In [9]:
import os 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import  sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Sequential
from keras.utils import pad_sequences
from sklearn.metrics import confusion_matrix
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',255)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
df = pd.read_csv("IMDB Dataset.csv")

In [19]:
df = df.iloc[:25000]

In [20]:
df['sentiment'].value_counts()

sentiment
negative    12526
positive    12474
Name: count, dtype: int64

In [21]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [22]:
df.drop_duplicates(inplace=True)

C:\Users\Manas\AppData\Local\Temp\ipykernel_16928\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [23]:
import re 
def remove_html(text):
    cleaned = re.sub(re.compile('<.*?>'),'',text)
    return cleaned

In [24]:
df['review'] = df['review'].apply(remove_html)

C:\Users\Manas\AppData\Local\Temp\ipykernel_16928\2166420511.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_html)


In [26]:
df['review'] = df['review'].apply(lambda x : x.lower())

C:\Users\Manas\AppData\Local\Temp\ipykernel_16928\1265626467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x : x.lower())


In [28]:
from nltk.corpus import stopwords
import nltk 
nltk.download('stopwords')
lt = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split()if item not in lt]).apply(lambda x: " ".join(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Manas\AppData\Local\Temp\ipykernel_16928\1097577974.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split()if item not in lt]).apply(lambda x: " ".join(x))


In [31]:
X = df.iloc[:,0:1]
y = df['sentiment']

In [32]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y=encoder.fit_transform(y)

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [36]:
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()